In [118]:
import pandas as pd
import json
import numpy as np

f = open('happenings.json')
happenings = json.load(f)
f.close()
#print(type(happenings))
#print(happenings[0])
happenings_converted = []
for i in happenings:
    happenings_converted.append(i[0:2])

#print(happenings_converted)
#for i in happenings:
#    print(i)
df_happenings = pd.DataFrame(happenings_converted, columns = ["Date", "Title"])
#print(df_happenings.head())

df_happenings['Date'] = pd.to_datetime(df_happenings['Date'])

#print(type(df_happenings['Date'][0]))
#print(df_happenings['Date'][0].isocalendar())

week = []
year = []
for i in df_happenings['Date']:
    #print(i) 
    year.append(i.isocalendar()[0])
    week.append(i.isocalendar()[1])
    
#print(year)
#print(week)
df_happenings["Year"] = year
df_happenings["Week"] = week

#print(df_happenings.head())

In [119]:
data = pd.read_csv("combined.csv", na_values = '-')
data.fillna(0, inplace = True)

#print(data)

#combined_total = data
combined_total = data.groupby(['Year', 'Week']).sum().reset_index()
combined_total.loc[:,'Age']="all"
combined_total = combined_total.reindex(columns = ["Year", "Week", "Age","Value","Value_Covid","Value_Covid_total"])
#print(combined_total)

data = data.append(combined_total, ignore_index=True).sort_values(by=['Year', 'Week']).reset_index(drop=True)
print(data)





      Year  Week    Age    Value  Value_Covid  Value_Covid_total
0     2015     1   0-34    200.0          0.0                0.0
1     2015     1  35-59   1637.0          0.0                0.0
2     2015     1  60-79   6665.0          0.0                0.0
3     2015     1    80+  10211.0          0.0                0.0
4     2015     1    all  18713.0          0.0                0.0
...    ...   ...    ...      ...          ...                ...
2345  2023    52   0-34    190.0          1.0              587.0
2346  2023    52  35-59   1394.0         15.0             8630.0
2347  2023    52  60-79   6743.0         80.0            53861.0
2348  2023    52    80+  13495.0        157.0           116992.0
2349  2023    52    all  21822.0        253.0           180070.0

[2350 rows x 6 columns]


In [128]:
year = 2023
kalenderwochen = [1,52]
lookback = 5
show_events = True

data_filtered = data.query("Year == @year and Week >= @kalenderwochen[0] and Week <= @kalenderwochen[1]")
#todo cumsum für alle altersgruppen für den gegebenen Zeitraum, sodass nicht nur der Grenzwert ganz am Ende ist sondern schon für den Verlauf

data_filtered_all = data_filtered[data_filtered["Age"] == "all"]
data_filtered_all["Value_Total"]= np.cumsum(data_filtered_all.loc[:,"Value"])
print(data_filtered_all)

data_filtered_young = data_filtered[data_filtered["Age"] == "0-34"]
data_filtered_young["Value_Total"]= np.cumsum(data_filtered_young.loc[:,"Value"])
print(data_filtered_young)

data_filtered_semi = data_filtered[data_filtered["Age"] == "35-59"]
data_filtered_semi["Value_Total"]= np.cumsum(data_filtered_semi.loc[:,"Value"])
print(data_filtered_semi)

data_filtered_old = data_filtered[data_filtered["Age"] == "60-79"]
data_filtered_old["Value_Total"]= np.cumsum(data_filtered_old.loc[:,"Value"])
print(data_filtered_old)

data_filtered_vold = data_filtered[data_filtered["Age"] == "80+"]
data_filtered_vold["Value_Total"]= np.cumsum(data_filtered_vold.loc[:,"Value"])
print(data_filtered_vold)


happenings_filtered = df_happenings.query("Year == @year and Week >= @kalenderwochen[0] and Week <= @kalenderwochen[1]")
#print(data_filtered)
#print(happenings_filtered)


      Year  Week  Age    Value  Value_Covid  Value_Covid_total  Value_Total
2094  2023     1  all  25384.0       1222.0           162537.0      25384.0
2099  2023     2  all  22695.0       1086.0           163623.0      48079.0
2104  2023     3  all  21215.0        929.0           164552.0      69294.0
2109  2023     4  all  20626.0        747.0           165299.0      89920.0
2114  2023     5  all  21081.0        681.0           165980.0     111001.0
2119  2023     6  all  20489.0        631.0           166611.0     131490.0
2124  2023     7  all  20978.0        542.0           167153.0     152468.0
2129  2023     8  all  20893.0        527.0           167680.0     173361.0
2134  2023     9  all  21041.0        582.0           168262.0     194402.0
2139  2023    10  all  21944.0        654.0           168916.0     216346.0
2144  2023    11  all  21806.0        607.0           169523.0     238152.0
2149  2023    12  all  21345.0        684.0           170207.0     259497.0
2154  2023  

c:\users\jean-luc busch\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\users\jean-luc busch\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\users\jean-luc busch\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try us

In [129]:
lookback_data = data.query("Year < @year and Year >= @year-@lookback and Week >= @kalenderwochen[0] and Week <= @kalenderwochen[1]")
print(lookback_data)

lookback_data_grouped = lookback_data.groupby(['Year','Age']).sum().reset_index().drop(columns = ['Week','Value_Covid_total'])
print(lookback_data_grouped)

expected_value_per_age_group = lookback_data_grouped.groupby(['Age']).mean().reset_index().drop("Year", axis = 1)
#print(expected_value_per_age_group)

std_expected_value_per_age_group = lookback_data_grouped.groupby(['Age']).std().reset_index().drop("Year", axis = 1)
#print(std_expected_value_per_age_group)

expected_value_per_age_group["Std"]=std_expected_value_per_age_group["Value"]
expected_value_per_age_group["Value_High"]=expected_value_per_age_group["Value"]+std_expected_value_per_age_group["Value"]
expected_value_per_age_group["Value_Low"]=expected_value_per_age_group["Value"]-std_expected_value_per_age_group["Value"]
expected_value_per_age_group.set_index("Age", inplace = True)
print(expected_value_per_age_group)


      Year  Week    Age    Value  Value_Covid  Value_Covid_total
785   2018     1   0-34    216.0          0.0                0.0
786   2018     1  35-59   1602.0          0.0                0.0
787   2018     1  60-79   6473.0          0.0                0.0
788   2018     1    80+  11051.0          0.0                0.0
789   2018     1    all  19342.0          0.0                0.0
...    ...   ...    ...      ...          ...                ...
2085  2022    52   0-34    237.0          1.0              528.0
2086  2022    52  35-59   1631.0         18.0             7950.0
2087  2022    52  60-79   8107.0        196.0            48369.0
2088  2022    52    80+  17309.0        484.0           104468.0
2089  2022    52    all  27284.0        699.0           161315.0

[1300 rows x 6 columns]
    Year    Age      Value  Value_Covid
0   2018   0-34    10427.0          0.0
1   2018  35-59    79344.0          0.0
2   2018  60-79   327110.0          0.0
3   2018    80+   535414.0         

In [130]:
lookback_data_new = data.query("Year < @year and Year >= @year-@lookback and Week >= @kalenderwochen[0] and Week <= @kalenderwochen[1]")
#print(lookback_data_new)

lookback_data_young = lookback_data_new[lookback_data_new["Age"]=="all"]
print(lookback_data_young)
#lookback_data_young["Value_Total"] = lookback_data_young.groupby(["Year","Week"])
print(lookback_data_young)
#lookback_data_young = lookback_data_new[lookback_data_new["Age"]=="0-34"]
#lookback_data_semi = lookback_data_new[lookback_data_new["Age"]=="35-59"]
#lookback_data_young = lookback_data_new[lookback_data_new["Age"]=="60-79"]
#lookback_data_young = lookback_data_new[lookback_data_new["Age"]=="80+"]

#lookback_data_new["Value_Total"] = lookback_data_new.groupby(["Year","Week"])['Value'].cumsum()
#print(lookback_data_new)



      Year  Week  Age    Value  Value_Covid  Value_Covid_total
789   2018     1  all  19342.0          0.0                0.0
794   2018     2  all  18770.0          0.0                0.0
799   2018     3  all  19187.0          0.0                0.0
804   2018     4  all  19171.0          0.0                0.0
809   2018     5  all  19558.0          0.0                0.0
...    ...   ...  ...      ...          ...                ...
2069  2022    48  all  22273.0        696.0           158034.0
2074  2022    49  all  24125.0        780.0           158814.0
2079  2022    50  all  25977.0        928.0           159742.0
2084  2022    51  all  28682.0        874.0           160616.0
2089  2022    52  all  27284.0        699.0           161315.0

[260 rows x 6 columns]
      Year  Week  Age    Value  Value_Covid  Value_Covid_total
789   2018     1  all  19342.0          0.0                0.0
794   2018     2  all  18770.0          0.0                0.0
799   2018     3  all  19187.0 

In [131]:
import plotly.graph_objects as go

fig_all = go.Figure()
fig_all.add_trace(go.Scatter(x = data_filtered_all["Week"], y = data_filtered_all["Value"], name = "all Cases per Week", line = dict(color = 'blue', width = 1,dash = 'dot')))
fig_all.add_trace(go.Scatter(x = data_filtered_all["Week"], y = np.cumsum(data_filtered_all["Value"]), name = "all Cases Total", line = dict(color = 'blue', width = 1)))





#fig_all.add_hline(y=expected_value_per_age_group.loc["all", "Value"], line_color = 'red', line_width = 1, name = "Expected all Cases Total")
fig_all.add_trace(go.Scatter(x = data_filtered_all["Week"],y = np.repeat(expected_value_per_age_group.loc["all","Value"],kalenderwochen[1]-kalenderwochen[0]+1), name = "expected Value", line = dict(color = 'red')))


#fig_all.add_hrect(y0=expected_value_per_age_group.loc["all","Value_Low"],y1=expected_value_per_age_group.loc["all","Value_High"],fillcolor = "red", line_width = 0, opacity = 0.2)


#print([expected_value_per_age_group.loc["all","Value"]]*52)
#print([expected_value_per_age_group.loc["all","Value_High"]]*52)
#print([expected_value_per_age_group.loc["all","Value_Low"]]*52)



fig_all.add_trace(go.Scatter(
    x = data_filtered_all["Week"].tolist() + data_filtered_all["Week"].tolist()[::-1] ,
    y = [expected_value_per_age_group.loc["all","Value_High"]]* (kalenderwochen[1]-kalenderwochen[0]+1) + [expected_value_per_age_group.loc["all","Value_Low"]]*(kalenderwochen[1]-kalenderwochen[0]+1),
    fill = 'toself',
    fillcolor = "red",
    line_color = "red",
    name = "Varianzbereich",
    opacity=0.2
))



#print(type(np.repeat(expected_value_per_age_group.loc["all","Value_High"],52) + np.repeat(expected_value_per_age_group.loc["all","Value_Low"],52)[::-1]))




fig_all.update_layout(title = f"Deaths  in {year} between week {kalenderwochen[0]} and week {kalenderwochen[1]}",
                        xaxis_title = "Week",
                        yaxis_title = "Deaths")


#print(happenings_filtered)

if show_events:
    for i in happenings_filtered.index:
        #print(happenings_filtered.loc[i,"Week"])
        #print(happenings_filtered.loc[i, "Title"])
        fig_all.add_vline(x = happenings_filtered.loc[i,"Week"], line_width = 2, line_color = "green", line_dash = "dash")

fig_all.show()

In [124]:


#print(data_filtered_young["Value"])
#print(np.cumsum(data_filtered_young["Value"]))

fig_young = go.Figure()
fig_young.add_trace(go.Scatter(x = data_filtered_young["Week"], y = data_filtered_young["Value"], name = "Young Cases per Week", line = dict(color = 'blue', width = 1,dash = 'dot')))
fig_young.add_trace(go.Scatter(x = data_filtered_young["Week"], y = np.cumsum(data_filtered_young["Value"]), name = "Young Cases Total", line = dict(color = 'blue', width = 1)))



#fig_young.add_hline(y=expected_value_per_age_group.loc["0-34", "Value"], line_color = 'red', line_width = 1, name = "Expected Young Cases Total")
fig_young.add_trace(go.Scatter(x = data_filtered_all["Week"],y = np.repeat(expected_value_per_age_group.loc["0-34","Value"],kalenderwochen[1]-kalenderwochen[0]+1), name = "expected Value", line = dict(color = 'red')))

#fig_young.add_hrect(y0=expected_value_per_age_group.loc["0-34","Value_Low"],y1=expected_value_per_age_group.loc["0-34","Value_High"],fillcolor = "red", line_width = 0, opacity = 0.2)
fig_young.add_trace(go.Scatter(
    x = data_filtered_all["Week"].tolist() + data_filtered_all["Week"].tolist()[::-1] ,
    y = [expected_value_per_age_group.loc["0-34","Value_Low"]]*(kalenderwochen[1]-kalenderwochen[0]+1) + [expected_value_per_age_group.loc["0-34","Value_High"]]*(kalenderwochen[1]-kalenderwochen[0]+1),
    fill = 'toself',
    fillcolor = "red",
    line_color = "red",
    name = "Varianzbereich",
    opacity=0.2
))


fig_young.update_layout(title = f"Deaths  in {year} between week {kalenderwochen[0]} and week {kalenderwochen[1]}",
                        xaxis_title = "Week",
                        yaxis_title = "Deaths")

if show_events:
    for i in happenings_filtered.index:
        #print(happenings_filtered.loc[i,"Week"])
        #print(happenings_filtered.loc[i, "Title"])
        fig_young.add_vline(x = happenings_filtered.loc[i,"Week"], line_width = 2, line_color = "green", line_dash = "dash")

fig_young.show()

#fig_total = go.Figure()
#fig_total.add_trace(go.Scatter())

In [125]:
fig_semi = go.Figure()
fig_semi.add_trace(go.Scatter(x = data_filtered_semi["Week"], y = data_filtered_semi["Value"], name = "semi Cases per Week", line = dict(color = 'blue', width = 1,dash = 'dot')))
fig_semi.add_trace(go.Scatter(x = data_filtered_semi["Week"], y = np.cumsum(data_filtered_semi["Value"]), name = "semi Cases Total", line = dict(color = 'blue', width = 1)))



#fig_semi.add_hline(y=expected_value_per_age_group.loc["35-59", "Value"], line_color = 'red', line_width = 1, name = "Expected semi Cases Total")
fig_semi.add_trace(go.Scatter(x = data_filtered_semi["Week"],y = np.repeat(expected_value_per_age_group.loc["35-59","Value"],kalenderwochen[1]-kalenderwochen[0]+1), name = "expected Value", line = dict(color = 'red')))

#fig_semi.add_hrect(y0=expected_value_per_age_group.loc["35-59","Value_Low"],y1=expected_value_per_age_group.loc["35-59","Value_High"],fillcolor = "red", line_width = 0, opacity = 0.2)

fig_semi.add_trace(go.Scatter(
    x = data_filtered_semi["Week"].tolist() + data_filtered_semi["Week"].tolist()[::-1] ,
    y = [expected_value_per_age_group.loc["35-59","Value_High"]]*(kalenderwochen[1]-kalenderwochen[0]+1) + [expected_value_per_age_group.loc["35-59","Value_Low"]]*(kalenderwochen[1]-kalenderwochen[0]+1),
    fill = 'toself',
    fillcolor = "red",
    line_color = "red",
    name = "Varianzbereich",
    opacity=0.2
))

fig_semi.update_layout(title = f"Deaths  in {year} between week {kalenderwochen[0]} and week {kalenderwochen[1]}",
                        xaxis_title = "Week",
                        yaxis_title = "Deaths")

if show_events:
    for i in happenings_filtered.index:
        #print(happenings_filtered.loc[i,"Week"])
        #print(happenings_filtered.loc[i, "Title"])
        fig_semi.add_vline(x = happenings_filtered.loc[i,"Week"], line_width = 2, line_color = "green", line_dash = "dash")

fig_semi.show()

In [126]:
fig_old = go.Figure()
fig_old.add_trace(go.Scatter(x = data_filtered_old["Week"], y = data_filtered_old["Value"], name = "old Cases per Week", line = dict(color = 'blue', width = 1,dash = 'dot')))
fig_old.add_trace(go.Scatter(x = data_filtered_old["Week"], y = np.cumsum(data_filtered_old["Value"]), name = "old Cases Total", line = dict(color = 'blue', width = 1)))



#fig_old.add_hline(y=expected_value_per_age_group.loc["60-79", "Value"], line_color = 'red', line_width = 1, name = "Expected old Cases Total")
fig_old.add_trace(go.Scatter(x = data_filtered_semi["Week"],y = np.repeat(expected_value_per_age_group.loc["60-79","Value"],kalenderwochen[1]-kalenderwochen[0]+1), name = "expected Value", line = dict(color = 'red')))

#fig_old.add_hrect(y0=expected_value_per_age_group.loc["60-79","Value_Low"],y1=expected_value_per_age_group.loc["60-79","Value_High"],fillcolor = "red", line_width = 0, opacity = 0.2)
fig_old.add_trace(go.Scatter(
    x = data_filtered_old["Week"].tolist() + data_filtered_old["Week"].tolist()[::-1] ,
    y = [expected_value_per_age_group.loc["60-79","Value_High"]]*(kalenderwochen[1]-kalenderwochen[0]+1) + [expected_value_per_age_group.loc["60-79","Value_Low"]]*(kalenderwochen[1]-kalenderwochen[0]+1),
    fill = 'toself',
    fillcolor = "red",
    line_color = "red",
    name = "Varianzbereich",
    opacity=0.2
))

fig_old.update_layout(title = f"Deaths  in {year} between week {kalenderwochen[0]} and week {kalenderwochen[1]}",
                        xaxis_title = "Week",
                        yaxis_title = "Deaths")

if show_events:
    for i in happenings_filtered.index:
        #print(happenings_filtered.loc[i,"Week"])
        #print(happenings_filtered.loc[i, "Title"])
        fig_old.add_vline(x = happenings_filtered.loc[i,"Week"], line_width = 2, line_color = "green", line_dash = "dash")

fig_old.show()

In [127]:
fig_vold = go.Figure()
fig_vold.add_trace(go.Scatter(x = data_filtered_vold["Week"], y = data_filtered_vold["Value"], name = "vold Cases per Week", line = dict(color = 'blue', width = 1,dash = 'dot')))
fig_vold.add_trace(go.Scatter(x = data_filtered_vold["Week"], y = np.cumsum(data_filtered_vold["Value"]), name = "vold Cases Total", line = dict(color = 'blue', width = 1)))


#fig_vold.add_hline(y=expected_value_per_age_group.loc["80+", "Value"], line_color = 'red', line_width = 1, name = "Expected vold Cases Total")
fig_vold.add_trace(go.Scatter(x = data_filtered_semi["Week"],y = np.repeat(expected_value_per_age_group.loc["80+","Value"],kalenderwochen[1]-kalenderwochen[0]+1), name = "expected Value", line = dict(color = 'red')))

#fig_vold.add_hrect(y0=expected_value_per_age_group.loc["80+","Value_Low"],y1=expected_value_per_age_group.loc["80+","Value_High"],fillcolor = "red", line_width = 0, opacity = 0.2)
fig_vold.add_trace(go.Scatter(
    x = data_filtered_vold["Week"].tolist() + data_filtered_vold["Week"].tolist()[::-1] ,
    y = [expected_value_per_age_group.loc["80+","Value_High"]]*(kalenderwochen[1]-kalenderwochen[0]+1) + [expected_value_per_age_group.loc["80+","Value_Low"]]*(kalenderwochen[1]-kalenderwochen[0]+1),
    fill = 'toself',
    fillcolor = "red",
    line_color = "red",
    name = "Varianzbereich",
    opacity=0.2
))


fig_vold.update_layout(title = f"Deaths in {year} between week {kalenderwochen[0]} and week {kalenderwochen[1]}",
                        xaxis_title = "Week",
                        yaxis_title = "Deaths")

if show_events:
    for i in happenings_filtered.index:
        #print(happenings_filtered.loc[i,"Week"])
        #print(happenings_filtered.loc[i, "Title"])
        fig_vold.add_vline(x = happenings_filtered.loc[i,"Week"], line_width = 2, line_color = "green", line_dash = "dash")

fig_vold.show()